## read dataset, vocabulary

In [1]:
import pickle

In [2]:
with open('./data/processed/WikiQACorpus_ELMO/train_token.pkl', 'rb') as f:
    trainset = pickle.load(f)
    
with open('./data/processed/WikiQACorpus_ELMO/dev_token.pkl', 'rb') as f:
    devset = pickle.load(f)
    
with open('./data/processed/WikiQACorpus_ELMO/test_token.pkl', 'rb') as f:
    testset = pickle.load(f)
    
trainset.keys()

dict_keys(['Q', 'A', 'L', 'N'])

In [3]:
dic = []
with open('./data/processed/WikiQACorpus/dic.pkl', 'rb') as f:
    dic = pickle.load(f)

print ('original dic size = ', len(dic))
dic['<S>'] = len(dic) + 1
dic['<\S>'] = len(dic) + 1

print ('add <S>, <\S>, dic size = ', len(dic))

with open('./data/processed/WikiQACorpus_ELMO/vocab.txt', 'w') as f:
    for key,val in dic.items():
        f.write( key + '\n')

original dic size =  30102
add <S>, <\S>, dic size =  30104


## Calculate EMLO embedding

In [4]:
import tensorflow as tf
import os
from bilm import TokenBatcher, BidirectionalLanguageModel, weight_layers, dump_token_embeddings    

In [5]:
W = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5'
O = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json'

In [6]:
os.system('mkdir ./data/ELMO_pretrain/')

256

In [7]:
cd ./data/ELMO_pretrain/

/home/dato/workspace/data_space/WikiQA/ELMO_pretrain


In [15]:
os.system('wget ' +  W)
os.system('wget ' +  O)

0

In [21]:
# raw_context = [
#     'Pretrained biLMs compute representations useful for NLP tasks .',
#     'They give state of the art performance for many tasks .'
# ]
# tokenized_context = [sentence.split() for sentence in raw_context]
# tokenized_question = [
#     ['What', 'are', 'biLMs', 'useful', 'for', '?'],
# ]

In [22]:
# import numpy as np
# np.shape(tokenized_context)

In [23]:
# # Create the vocabulary file with all unique tokens and
# # the special <S>, </S> tokens (case sensitive).
# all_tokens = set(['<S>', '</S>'] + tokenized_question[0])
# for context_sentence in tokenized_context:
#     for token in context_sentence:
#         all_tokens.add(token)
# vocab_file = 'vocab_small.txt'
# with open(vocab_file, 'w') as fout:
#     fout.write('\n'.join(all_tokens))

In [13]:
pwd

'/home/dato/workspace/data_space/WikiQA/ELMO_pretrain'

In [16]:
os.system('ln -s ../../ELMO_pretrain/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json ELMO_options.json')
os.system('ln -s ../../ELMO_pretrain/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5 ELMO_weights.hdf5')

0

In [17]:
os.system('mv ./ELMO_* ../processed/WikiQACorpus_ELMO/')

0

In [18]:
# Location of pretrained LM.  Here we use the test fixtures.
datadir = '/home/dato/data_space/WikiQA/processed/WikiQACorpus_ELMO/'
options_file = os.path.join(datadir, 'ELMO_options.json')
weight_file = os.path.join(datadir, 'ELMO_weights.hdf5')
vocab_file =  os.path.join(datadir, 'vocab.txt')
token_embedding_file =  os.path.join(datadir, 'WikiQA_elmo_token_embeddings.hdf5')

'/home/dato/data_space/WikiQA/processed/WikiQACorpus_ELMO/WikiQA_elmo_token_embeddings.hdf5'

In [19]:
# Dump the token embeddings to a file. Run this once for your dataset.
dump_token_embeddings(
    vocab_file, options_file, weight_file, token_embedding_file
)
tf.reset_default_graph()

Instructions for updating:
Use the `axis` argument instead
USING SKIP CONNECTIONS


In [7]:
## Now we can do inference.
# Create a TokenBatcher to map text to token ids.
batcher = TokenBatcher(vocab_file)

# Input placeholders to the biLM.
context_token_ids = tf.placeholder('int32', shape=(None, None))
question_token_ids = tf.placeholder('int32', shape=(None, None))

# Build the biLM graph.
bilm = BidirectionalLanguageModel(
    options_file,
    weight_file,
    use_character_inputs=False,
    embedding_weight_file=token_embedding_file
)

In [8]:
# Get ops to compute the LM embeddings.
context_embeddings_op = bilm(context_token_ids)
question_embeddings_op = bilm(question_token_ids)

# Get an op to compute ELMo (weighted average of the internal biLM layers)
# Our SQuAD model includes ELMo at both the input and output layers
# of the task GRU, so we need 4x ELMo representations for the question
# and context at each of the input and output.
# We use the same ELMo weights for both the question and context
# at each of the input and output.
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)
with tf.variable_scope('', reuse=True):
    # the reuse=True scope reuses weights from the context for the question
    elmo_question_input = weight_layers(
        'input', question_embeddings_op, l2_coef=0.0
    )

elmo_context_output = weight_layers(
    'output', context_embeddings_op, l2_coef=0.0
)

with tf.variable_scope('', reuse=True):
    # the reuse=True scope reuses weights from the context for the question
    elmo_question_output = weight_layers(
        'output', question_embeddings_op, l2_coef=0.0
    )

USING SKIP CONNECTIONS
USING SKIP CONNECTIONS
Instructions for updating:
Use the `axis` argument instead


In [10]:
tokenized_context

NameError: name 'tokenized_context' is not defined

In [22]:
context_ids = batcher.batch_sentences(tokenized_context)

In [43]:
print (tokenized_context)
print (np.shape(tokenized_context))
print (np.shape(tokenized_context[0]))
print (np.shape(tokenized_context[1]))

[['Pretrained', 'biLMs', 'compute', 'representations', 'useful', 'for', 'NLP', 'tasks', '.'], ['They', 'give', 'state', 'of', 'the', 'art', 'performance', 'for', 'many', 'tasks', '.']]
(2,)
(9,)
(11,)


In [38]:
print (tokenized_question)
print (np.shape(tokenized_question))

[['What', 'are', 'biLMs', 'useful', 'for', '?']]
(1, 6)


In [39]:
# Create batches of data.
context_ids = batcher.batch_sentences(tokenized_context)
question_ids = batcher.batch_sentences(tokenized_question)

In [40]:
print (context_ids)
print (np.shape(context_ids))

[[30103     0     0 16855  6219 26728  7921     0 15576   294     0     0
      0]
 [30103   205 11348 25997 14030  9732 29360  4412  7921 15791 15576   294
      0]]
(2, 13)


In [41]:
print (question_ids)
print (np.shape(question_ids))

[[30103 12290 29366     0 26728  7921 18591     0]]
(1, 8)


In [11]:
with tf.Session() as sess:
    # It is necessary to initialize variables once before running inference.
    sess.run(tf.global_variables_initializer())

    # Create batches of data.
    context_ids = batcher.batch_sentences(tokenized_context)
    question_ids = batcher.batch_sentences(tokenized_question)

    # Compute ELMo representations (here for the input only, for simplicity).
    elmo_context_input_, elmo_question_input_ = sess.run(
        [elmo_context_input['weighted_op'], elmo_question_input['weighted_op']],
        feed_dict={context_token_ids: context_ids,
                   question_token_ids: question_ids}
    )

In [20]:
tokenized_question

[['What', 'are', 'biLMs', 'useful', 'for', '?']]

In [25]:
np.shape(tokenized_question)

(1, 6)

In [26]:
tokenized_context

[['Pretrained',
  'biLMs',
  'compute',
  'representations',
  'useful',
  'for',
  'NLP',
  'tasks',
  '.'],
 ['They',
  'give',
  'state',
  'of',
  'the',
  'art',
  'performance',
  'for',
  'many',
  'tasks',
  '.']]

In [27]:
np.shape(tokenized_context)

(2,)

In [39]:
np.shape(elmo_question_input_)

(1, 6, 1024)

In [40]:
np.shape(elmo_context_input_)

(2, 11, 1024)

In [41]:
elmo_context_input_[0]

array([[ 0.02462964,  0.16383222,  0.2964051 , ..., -0.12922229,
        -0.2930111 ,  0.13189684],
       [-0.57287556, -0.07158805, -0.24308902, ..., -0.11735731,
        -0.2677825 , -0.06677829],
       [-0.8419022 ,  0.2892721 , -0.3882361 , ..., -0.21937613,
        -0.16024263, -0.09764783],
       ...,
       [-0.03325855,  0.00599727, -0.12481157, ...,  0.27898324,
         0.55338466, -0.18398118],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [42]:
elmo_context_input_[1]

array([[ 0.24442434, -0.3860852 ,  0.09052717, ...,  0.12260772,
        -0.01382168, -0.06875429],
       [-0.338963  ,  0.02178424,  0.22192626, ..., -0.35271585,
         0.26657373, -0.27544504],
       [-0.35692668, -0.32613516,  0.2805973 , ...,  0.00421192,
         0.09699225,  0.09507689],
       ...,
       [-0.06032594, -0.07026625,  0.0918818 , ..., -0.2848844 ,
        -0.19069   , -0.19516918],
       [ 0.06933622, -0.30043387,  0.1031775 , ..., -0.5641655 ,
         0.21887177, -0.06783633],
       [ 0.04119237, -0.06924646,  0.01466054, ...,  0.27898324,
         0.55338466, -0.18398118]], dtype=float32)